<h1 style="text-align: center">
<strong>UNALM: PRÁCTICA CALIFICADA 3</strong>
</h1>

**Integrantes:**
1. Pichardo Diestra, Oliverio
2. Ortiz Arone, Víctor Alejandro
3. Llamoca León, Israel

In [1]:
# Tu clave de API de Spoonacular
api_key = "1a60313c31a84c6d80a50e336bd18421"

In [2]:
import requests
# ID de la receta a la que quieres encontrar recetas similares
recipe_id = 47746

# Realizar una llamada a la API para obtener recetas similares
url = f"https://api.spoonacular.com/recipes/{recipe_id}/similar"
query_params = {"apiKey": api_key}
response = requests.get(url, params=query_params).json()

response

[{'id': 456541,
  'imageType': 'jpg',
  'title': 'Spiced Apples',
  'readyInMinutes': 45,
  'servings': 8,
  'sourceUrl': 'http://allrecipes.com/Recipe/Spiced-Apples/'},
 {'id': 403407,
  'imageType': 'jpg',
  'title': 'Glazed Spiced Apples',
  'readyInMinutes': 25,
  'servings': 10,
  'sourceUrl': 'http://www.tasteofhome.com/recipes/glazed-spiced-apples'},
 {'id': 48263,
  'imageType': 'jpg',
  'title': 'Chicken & Spiced Apples',
  'readyInMinutes': 35,
  'servings': 6,
  'sourceUrl': 'http://www.eatingwell.com/recipes/chicken_spiced_apples.html'},
 {'id': 313917,
  'imageType': 'jpeg',
  'title': 'Spiced Apples and Raisins',
  'readyInMinutes': 38,
  'servings': 4,
  'sourceUrl': 'http://www.foodnetwork.com/recipes/tyler-florence/spiced-apples-and-raisins-recipe.html'},
 {'id': 403413,
  'imageType': 'jpg',
  'title': 'Spiced Baked Apples',
  'readyInMinutes': 45,
  'servings': 2,
  'sourceUrl': 'http://www.tasteofhome.com/Recipes/spiced-baked-apples'},
 {'id': 586610,
  'imageType':

In [3]:
import pandas as pd
pd.DataFrame(response)

,id,imageType,title,readyInMinutes,servings,sourceUrl
0,456541,jpg,Spiced Apples,45,8,http://allrecipes.com/Recipe/Spiced-Apples/
1,403407,jpg,Glazed Spiced Apples,25,10,http://www.tasteofhome.com/recipes/glazed-spic...
2,48263,jpg,Chicken & Spiced Apples,35,6,http://www.eatingwell.com/recipes/chicken_spic...
3,313917,jpeg,Spiced Apples and Raisins,38,4,http://www.foodnetwork.com/recipes/tyler-flore...
4,403413,jpg,Spiced Baked Apples,45,2,http://www.tasteofhome.com/Recipes/spiced-bake...
5,586610,jpg,Vanilla Spiced Apples,35,8,http://www.nutmegnanny.com/2011/05/02/vanilla-...
6,930836,jpeg,Spiced Chicken with Apples,40,4,http://www.foodnetwork.com/recipes/food-networ...
7,151791,jpg,Spiced Waffles with Caramelized Apples,45,4,http://www.epicurious.com/recipes/food/views/S...
